# Issue 132 Notebook Toaletter nära SAT
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb)
* [Issue 132](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132)

Se liknande lösning för Roslagsleden
* nu har vi SAT = wikidata [Q131318799](https://www.wikidata.org/wiki/Q131318799)
* "leden" sitter inte ihop utan varje ö har sitt segment

Output 
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_30_04_22.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_30_04_22.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html)


version 0.1
* unary_union --> union_all
* start updating OSM with pictures from https://commons.wikimedia.org/wiki/Category:SAT_Todo
* also count number of seats ie. toilets:number
* Fetch SAT etapper via Wikidata
* Fetch geometries per relation via Overpass (no zip-mismatch)
* Buffer 200 m (Shapely ≥2 with union_all())
* Fetch toilets as nodes/ways/relations via nwr[...] out center;
* Count toilets via toilets:number (fallback male/female/unisex → else 1)
* Join to nearest etapp, build summary (CSV)
* Build a Folium map with etapper, 200 m buffer, and toilet markers
* Export toilets within 200 m as GeoJSON + CSV and save the map HTML

In [13]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-08-17 19:25:14


In [15]:
# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os, re, requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, MultiLineString, Point, mapping
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup
from datetime import datetime

# =========================
# 1) Hämta SAT-etapper via Wikidata
# =========================
print("🔍 Hämtar SAT-etapper från Wikidata...")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "id": r["osmid"]["value"],
        "label": r.get("itemLabel", {}).get("value", ""),
        "island": r.get("islandLabel", {}).get("value", "")
    })
osm_ids = [e['id'] for e in etapper]
print(f"✅ Hittade {len(osm_ids)} etapper med OSM-relationer")

# =========================
# 2) Hämta geometrier per relation via Overpass
# =========================
print("📡 Hämtar geometrier från Overpass (per relation)...")
overpass_url = "http://overpass-api.de/api/interpreter"
geom_per_rel = {}
all_lines = []

for rel_id in osm_ids:
    q = f"""
    [out:json][timeout:60];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    r = requests.post(overpass_url, data={"data": q})
    if r.status_code != 200:
        print(f"⚠️ Fel för relation {rel_id}: {r.text[:200]}...")
        continue
    rel_geoms = []
    for el in r.json().get("elements", []):
        if el.get("type") == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 2:
                line = LineString(coords)
                rel_geoms.append(line)
                all_lines.append(line)
    if rel_geoms:
        geom_per_rel[rel_id] = rel_geoms

if not all_lines:
    raise ValueError("Inga geometrier hittades från OSM-relationer kopplade via Wikidata.")

# Bygg MultiLineString/LineString per etapp
meta_rows, geom_rows = [], []
for meta in etapper:
    geoms = geom_per_rel.get(meta["id"])
    if not geoms:
        print(f"⚠️ Saknar geometri för {meta['label']} (rel {meta['id']}) – hoppar över.")
        continue
    meta_rows.append(meta)
    geom_rows.append(MultiLineString(geoms) if len(geoms) > 1 else geoms[0])

gdf_trail = gpd.GeoDataFrame(geometry=all_lines, crs="EPSG:4326")
meta_gdf = gpd.GeoDataFrame(meta_rows, geometry=geom_rows, crs="EPSG:4326")
print(f"🧭 Etapper med geometri: {len(meta_gdf)}")

# =========================
# 3) Buffert 200 m (Shapely 2: union_all)
# =========================
print("🧮 Skapar 200 m-buffert...")
buffer_utm = gdf_trail.to_crs(3006).buffer(200)   # 200 m i SWEREF 99 TM
buffer_wgs84 = buffer_utm.to_crs(4326)            # tillbaka till WGS84
buffer_union = buffer_wgs84.union_all()           # EN (multi)polygon

# =========================
# 4) Hämta toaletter (noder/ways/relationer) via nwr + out center
# =========================
def _parse_int(v):
    if v is None:
        return None
    m = re.search(r"\d+", str(v))
    return int(m.group()) if m else None

def toilets_count_from_tags(tags: dict) -> int:
    """
    Prioritet:
      1) toilets:number
      2) sum(male:toilets, female:toilets, unisex:toilets)
      3) default = 1
    """
    n = _parse_int(tags.get("toilets:number"))
    if n is not None:
        return n
    parts = [
        _parse_int(tags.get("male:toilets")),
        _parse_int(tags.get("female:toilets")),
        _parse_int(tags.get("unisex:toilets")),
    ]
    parts = [p for p in parts if p is not None]
    if parts:
        return sum(parts)
    return 1

bbox = gdf_trail.total_bounds  # [minx, miny, maxx, maxy]
q_toilets = f"""
[out:json][timeout:60];
nwr["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out center;
"""
print("🚽 Hämtar toaletter (nwr) från Overpass...")
r = requests.post(overpass_url, data={"data": q_toilets})
elements = r.json().get("elements", [])
toilets = []
for el in elements:
    tags = el.get("tags", {})
    typ = el.get("type")
    if typ == "node":
        lon, lat = el["lon"], el["lat"]
    else:
        center = el.get("center")
        if not center:
            continue
        lon, lat = center["lon"], center["lat"]
    toilets.append({
        "geometry": Point(lon, lat),
        "tags": tags,
        "id": el["id"],
        "osm_type": typ,
        "toilets_num": toilets_count_from_tags(tags),
    })

gdf_toilets = gpd.GeoDataFrame(toilets, crs="EPSG:4326")
print(f"✅ Hittade {len(gdf_toilets)} toalett-objekt inom bbox")

# =========================
# 5) Filtrera till de som ligger inom/vid 200 m-bufferten
# =========================
in_range = gdf_toilets[gdf_toilets.geometry.covered_by(buffer_union)]  # covered_by inkluderar gränsen
print(f"✅ {len(in_range)} toalett-objekt inom/vid 200 m")

# =========================
# 6) Närmaste etapp per toalett
# =========================
meta_utm = meta_gdf.to_crs(3006)
toilets_utm = in_range.to_crs(3006)
joined = gpd.sjoin_nearest(
    toilets_utm,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

# =========================
# 7) Summary (räkna toilets:number)
# =========================
summary = (
    joined.assign(toilets_num=joined["toilets_num"].fillna(1))
    .groupby(["label", "island"], as_index=False)
    .agg(
        sites=("geometry", "count"),           # antal OSM-objekt (noder/ways/relationer)
        toilets_total=("toilets_num", "sum"),  # totalt antal toaletter
        avg_distance_m=("distance_m", "mean"),
    )
    .assign(avg_toilets_per_site=lambda df: df["toilets_total"] / df["sites"])
    .sort_values(["toilets_total", "sites"], ascending=[False, False])
)
print("📊 Summary:")
print(summary.head(1000))

# =========================
# 8) Spara filer + Folium-karta
# =========================
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
os.makedirs("../kartor", exist_ok=True)

# Spara summary
summary_csv = f"../kartor/sat_toaletter_summary_{timestamp}.csv"
summary.to_csv(summary_csv, index=False)

# Spara toaletter inom 200 m (GeoJSON + CSV)
toilets_geojson = f"../kartor/sat_toaletter_inrange_{timestamp}.geojson"
toilets_csv = f"../kartor/sat_toaletter_inrange_{timestamp}.csv"
in_range[["id", "osm_type", "toilets_num", "tags", "geometry"]].to_file(toilets_geojson, driver="GeoJSON")
in_range.drop(columns="geometry").to_csv(toilets_csv, index=False)

# Bygg karta
center = gdf_trail.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=9, control_scale=True)

# Etapper
colors = [
    "blue","green","purple","orange","darkred","cadetblue","lightgray","darkblue",
    "darkgreen","pink","lightblue","lightgreen","gray","black","beige","lightred"
]
for i, row in meta_gdf.reset_index(drop=True).iterrows():
    color = colors[i % len(colors)]
    popup = f"<b>{row['label']}</b><br>Ö: {row['island']}"
    folium.GeoJson(
        data=mapping(row.geometry),
        name=row["label"],
        style_function=lambda x, c=color: {"color": c, "weight": 3}
    ).add_child(folium.Popup(popup, max_width=350)).add_to(m)

# Buffert
folium.GeoJson(
    data=mapping(buffer_union),
    name="200 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

# Toaletter
toilets_fg = FeatureGroup(name="Toaletter inom 200 m")
for _, r in joined.to_crs(4326).iterrows():
    tags = r.get("tags", {}) or {}
    osm_url = f"https://www.openstreetmap.org/{'node' if r['osm_type']=='node' else 'way' if r['osm_type']=='way' else 'relation'}/{r['id']}"
    popup_html = f"""
    <b><a href="{osm_url}" target="_blank">OSM-objekt ({r['osm_type']})</a></b><br>
    Etapp: <b>{r.get('label','')}</b> (Ö: {r.get('island','')})<br>
    Avstånd: ~{round(r.get('distance_m', 0), 1)} m<br>
    Antal toaletter: <b>{int(r.get('toilets_num', 1))}</b><br>
    """
    if "opening_hours" in tags:
        popup_html += f"Öppettider: {tags['opening_hours']}<br>"
    if "wheelchair" in tags:
        popup_html += f"Rullstol: {tags['wheelchair']}<br>"
    if "toilets:disposal" in tags:
        popup_html += f"Avfall: {tags['toilets:disposal']}<br>"

    Marker(
        location=[r.geometry.y, r.geometry.x],
        popup=Popup(popup_html, max_width=320),
        icon=Icon(color="blue", icon="info-sign")
    ).add_to(toilets_fg)

toilets_fg.add_to(m)
LayerControl(collapsed=False).add_to(m)

map_html = f"../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
m.save(map_html)

print("✅ Klart!")
print(f"• Summary CSV: {summary_csv}")
print(f"• Toilets GeoJSON: {toilets_geojson}")
print(f"• Toilets CSV: {toilets_csv}")
print(f"• Karta: {map_html}")


🔍 Hämtar SAT-etapper från Wikidata...
✅ Hittade 20 etapper med OSM-relationer
📡 Hämtar geometrier från Overpass (per relation)...
🧭 Etapper med geometri: 20
🧮 Skapar 200 m-buffert...
🚽 Hämtar toaletter (nwr) från Overpass...
✅ Hittade 721 toalett-objekt inom bbox
✅ 96 toalett-objekt inom/vid 200 m
📊 Summary:
            label     island  sites  toilets_total  avg_distance_m  \
10    SAT Nåttarö    Nåttarö     10             28       67.357161   
1    SAT Finnhamn   Finnhamn     16             17       29.151231   
2   SAT Fjärdlång  Fjärdlång     10             14       50.783603   
4      SAT Grinda     Grinda     12             12       31.417178   
0     SAT Arholma    Arholma      9             11       39.197231   
14        SAT Utö        Utö      8              8       17.478826   
12       SAT Rånö       Rånö      6              7       32.673998   
5    SAT Ingmarsö   Ingmarsö      5              5       24.822614   
13   SAT Sandhamn     Sandön      4              4       16.

In [16]:
import requests
import folium
from SPARQLWrapper import SPARQLWrapper, JSON
from shapely.geometry import LineString

# Get SAT trail sections from Wikidata (label + OSM ID + image)
def get_sat_sections():
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = """
    SELECT ?section ?sectionLabel ?osmRelation ?image WHERE {
      ?section wdt:P361 wd:Q131318799;     # part of SAT
               wdt:P402 ?osmRelation.      # OSM relation ID
      OPTIONAL { ?section wdt:P18 ?image. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    ORDER BY ?sectionLabel
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    sections = []
    for result in results["results"]["bindings"]:
        label = result["sectionLabel"]["value"]
        osm_id = result["osmRelation"]["value"]
        image = result.get("image", {}).get("value", None)
        sections.append({"label": label, "osm_id": osm_id, "image": image})
    return sections

# Overpass query per relation
def fetch_osm_geometry(relation_id):
    query = f"""
    [out:json][timeout:25];
    relation({relation_id});
    (._; >>;);
    out geom;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data=query)
    data = response.json()

    lines = []
    for el in data["elements"]:
        if el["type"] == "way" and "geometry" in el:
            coords = [(pt["lat"], pt["lon"]) for pt in el["geometry"]]
            lines.append(coords)
    return lines

# Create the map
m = folium.Map(location=[59.7, 19.1], zoom_start=10, tiles="OpenStreetMap")

# Main logic
sections = get_sat_sections()
for section in sections:
    label = section["label"]
    osm_id = section["osm_id"]
    image = section["image"]

    try:
        lines = fetch_osm_geometry(osm_id)
        all_coords = []
        for coords in lines:
            folium.PolyLine(coords, color="blue", weight=4, tooltip=label).add_to(m)
            all_coords.extend(coords)
        
        # Add a marker with image at trail midpoint
        if all_coords:
            mid_idx = len(all_coords) // 2
            lat, lon = all_coords[mid_idx]
            popup_html = f"<b>{label}</b>"
            if image:
                popup_html += f"<br><img src='{image}' width='200'>"
            folium.Marker(
                location=(lat, lon),
                popup=folium.Popup(popup_html, max_width=250),
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(m)
    except Exception as e:
        print(f"Error fetching OSM relation {osm_id}: {e}")

# Display map
m



In [17]:
import requests
import folium
from SPARQLWrapper import SPARQLWrapper, JSON

# Get SAT trail sections from Wikidata
def get_sat_sections():
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = """
    SELECT ?section ?sectionLabel ?osmRelation ?image ?category WHERE {
      ?section wdt:P361 wd:Q131318799;
               wdt:P402 ?osmRelation.
      OPTIONAL { ?section wdt:P18 ?image. }
      OPTIONAL { ?section wdt:P373 ?category. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    ORDER BY ?sectionLabel
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    sections = []
    for result in results["results"]["bindings"]:
        label = result["sectionLabel"]["value"]
        osm_id = result["osmRelation"]["value"]
        wikidata_id = result["section"]["value"].split("/")[-1]
        image = result.get("image", {}).get("value", None)
        category = result.get("category", {}).get("value", None)
        sections.append({
            "label": label,
            "osm_id": osm_id,
            "image": image,
            "wikidata_id": wikidata_id,
            "category": category
        })
    return sections

# Overpass query per relation
def fetch_osm_geometry(relation_id):
    query = f"""
    [out:json][timeout:25];
    relation({relation_id});
    (._; >>;);
    out geom;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data=query)
    data = response.json()

    lines = []
    for el in data["elements"]:
        if el["type"] == "way" and "geometry" in el:
            coords = [(pt["lat"], pt["lon"]) for pt in el["geometry"]]
            lines.append(coords)
    return lines

# Create the map
m = folium.Map(location=[59.7, 19.1], zoom_start=10, tiles="OpenStreetMap")

# Main logic
sections = get_sat_sections()
for section in sections:
    label = section["label"]
    osm_id = section["osm_id"]
    image = section["image"]
    wikidata_id = section["wikidata_id"]
    category = section["category"]

    try:
        lines = fetch_osm_geometry(osm_id)
        all_coords = []
        for coords in lines:
            folium.PolyLine(coords, color="blue", weight=4, tooltip=label).add_to(m)
            all_coords.extend(coords)
        
        if all_coords:
            mid_idx = len(all_coords) // 2
            lat, lon = all_coords[mid_idx]
            popup_html = f"<b>{label}</b><br>"
            if image:
                popup_html += f"<img src='{image}' width='200'><br>"
            popup_html += (
                f"<a href='https://www.openstreetmap.org/relation/{osm_id}' target='_blank'>OSM</a> | "
                f"<a href='https://www.wikidata.org/wiki/{wikidata_id}' target='_blank'>Wikidata</a>"
            )
            if category:
                commons_url = f"https://commons.wikimedia.org/wiki/Category:{category.replace(' ', '_')}"
                popup_html += f" | <a href='{commons_url}' target='_blank'>Commons</a>"
            folium.Marker(
                location=(lat, lon),
                popup=folium.Popup(popup_html, max_width=250),
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(m)
    except Exception as e:
        print(f"Error fetching OSM relation {osm_id}: {e}")

# Display map
m


In [18]:
# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import requests, os
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup
from datetime import datetime

# === Step 1: Get SAT trail sections from Wikidata ===
print("🔍 Hämtar SAT-etapper från Wikidata...")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "id": r["osmid"]["value"],
        "label": r.get("itemLabel", {}).get("value", ""),
        "island": r.get("islandLabel", {}).get("value", "")
    })

# === Step 2: Fetch trail geometries from Overpass ===
print(f"📡 Hämtar geometrier för {len(etapper)} OSM-relationer...")
overpass_url = "http://overpass-api.de/api/interpreter"
geom_per_rel = defaultdict(list)
all_lines = []

for e in etapper:
    rel_id = e['id']
    query = f"""
    [out:json];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    r = requests.post(overpass_url, data={"data": query})
    if r.status_code != 200:
        print(f"⚠️ Fel för relation {rel_id}")
        continue
    for el in r.json()["elements"]:
        if el["type"] == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            geom = LineString(coords)
            geom_per_rel[rel_id].append(geom)
            all_lines.append(geom)

gdf_trail = gpd.GeoDataFrame(geometry=all_lines, crs="EPSG:4326")
geometries = [
    MultiLineString(geom_per_rel[e["id"]]) if len(geom_per_rel[e["id"]]) > 1 else geom_per_rel[e["id"]][0]
    for e in etapper
]
meta_gdf = gpd.GeoDataFrame(etapper, geometry=geometries, crs="EPSG:4326")

# === Step 3: Buffer trail 200 meters ===
print("🧮 Skapar 200 meters buffert...")
buffer_utm = gdf_trail.to_crs(3006).buffer(200)
buffer_wgs84 = buffer_utm.to_crs(4326) 
# slå ihop alla buffertpolygoner till en (multi)polygon
buffer_union = buffer_wgs84.union_all() 

# === Step 4: Query OSM toilets ===
bbox = gdf_trail.total_bounds
query_toilets = f"""
[out:json][timeout:25];
node["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out;
"""
print("🚽 Hämtar toaletter från Overpass API...")
r = requests.post(overpass_url, data={"data": query_toilets})
toilets_data = r.json()

toilets = []
for el in toilets_data['elements']:
    if el['type'] == 'node':
        pt = Point(el["lon"], el["lat"])
        toilets.append({"geometry": pt, "tags": el.get("tags", {}), "id": el["id"]})

gdf_toilets = gpd.GeoDataFrame(toilets, crs="EPSG:4326")
print(f"✅ Hittade {len(gdf_toilets)} toaletter inom bbox")

# === Step 5: Filter to toilets within 200m buffer ===
in_range = gdf_toilets[gdf_toilets.geometry.within(buffer_union)]
print(f"✅ {len(in_range)} toaletter ligger inom 200m från leden")

# === Step 6: Match each toilet to closest trail section ===
toilets_utm = in_range.to_crs(3006)
meta_utm = meta_gdf.to_crs(3006)
joined = gpd.sjoin_nearest(toilets_utm, meta_utm[["label", "island", "geometry"]],
                           how="left", distance_col="distance_m")
joined = joined.to_crs(4326)

# === Step 7: Build the interactive map ===
m = folium.Map(location=[gdf_trail.geometry[0].centroid.y, gdf_trail.geometry[0].centroid.x], zoom_start=9)

# Add trail geometries
for _, row in meta_gdf.iterrows():
    folium.GeoJson( mapping(row.geometry), name=row["label"],
                   tooltip=row["label"],
                   style_function=lambda x: {"color": "blue", "weight": 3}).add_to(m)

# Add buffer 
folium.GeoJson(
    data=mapping(buffer_union),
    name="200m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

# Add toilet markers
good_group = FeatureGroup(name="Godkända toaletter")
warn_group = FeatureGroup(name="Varningar / Ofullständiga")

for _, row in joined.iterrows():
    tags = row['tags']
    el_id = row["id"]
    osm_url = f"https://www.openstreetmap.org/node/{el_id}"
    popup_text = f"<b><a href='{osm_url}' target='_blank'>OSM objekt</a></b><br>"

    # Metadata
    if "image" in tags:
        popup_text += f"<img src='{tags['image']}' width='200'><br>"
    if "opening_hours" in tags:
        popup_text += f"<b>Öppettider:</b> {tags['opening_hours']}<br>"
    if "wheelchair" in tags:
        popup_text += f"<b>Rullstol:</b> {tags['wheelchair']}<br>"
    if "toilets:disposal" in tags:
        popup_text += f"<b>Avfallshantering:</b> {tags['toilets:disposal']}<br>"

    popup_text += "<hr style='margin:5px 0;'><ul style='padding-left:18px;'>"

    required = ["amenity", "access", "unisex", "toilets:disposal", "wheelchair"]
    recommended = ["toilets:handwashing", "toilets:paper_supply", "opening_hours", "fee", "operator", "description"]
    missing_req = [t for t in required if t not in tags]
    missing_opt = [t for t in recommended if t not in tags]

    for t in missing_req:
        popup_text += f"<li><b>{t}</b> saknas</li>"
    for t in missing_opt:
        popup_text += f"<li>{t} saknas</li>"
    popup_text += "</ul><i>Tips: kontrollera viktiga attribut</i>"

    icon_color = "red" if missing_req else "blue"
    Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=Popup(popup_text, max_width=300),
        icon=Icon(color=icon_color, icon="info-sign")
    ).add_to(warn_group if missing_req else good_group)
center = gdf_trail.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=9)
m.add_child(good_group)
m.add_child(warn_group)
LayerControl().add_to(m)

# === Step 8: Save to HTML ===
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
filename = f"Issue_132_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
output_path = f"../kartor/{filename}"
os.makedirs("../kartor", exist_ok=True)
m.save(output_path)

print(f"✅ Karta sparad: {output_path}")


🔍 Hämtar SAT-etapper från Wikidata...
📡 Hämtar geometrier för 20 OSM-relationer...
🧮 Skapar 200 meters buffert...
🚽 Hämtar toaletter från Overpass API...
✅ Hittade 593 toaletter inom bbox
✅ 76 toaletter ligger inom 200m från leden
✅ Karta sparad: ../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html


In [7]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-08-17 19:15:36
Total time elapsed: 70.83 seconds
